In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tabulate import tabulate
import pandas_profiling
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import warnings
from IPython.display import IFrame
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline
warnings.filterwarnings('ignore')

df = pd.read_excel("../data/info_satisfaccion_trabajo.xlsx")
df.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


# __EDA with pandas_profiling__ 

[EDA with pandas_profiling](https://davidcamilo0710.github.io/Coink)


In [2]:
# # Generate report using pandas-profiling
# report = df.profile_report()

# # Save the report to an HTML file
# report.to_file("report.html")

# # View the report in the notebook
# IFrame(src='report.html', width=1000, height=600)

The report shows that the dataset contains 1470 observations and 35 variables, with 15 being numeric, 3 boolean, and 17 categorical. No missing cells or duplicate rows were found in the dataset.

The Alerts highlight some important characteristics of the dataset, including the presence of constant variables (EmployeeCount, Over18, StandardHours), and high correlations between several numeric variables, such as Age with TotalWorkingYears, MonthlyIncome with TotalWorkingYears and one other variable, PercentSalaryHike with PerformanceRating, among others. There is also high correlation between categorical variables like Department and EducationField, and JobRole and Department.

Overall, the pandas_profiling report provides valuable information about the dataset characteristics, including the distribution and type of variables, as well as relationships and correlations between them. It also detects constant variables and missing or zero values. This information will be useful for developing the requested supervised learning models and save some time.

### Resumamos el dataset

In [3]:
def dataframe_summary(df):
    # Number of rows and columns
    summary = []
    summary.append(["Number of rows and columns:", f"{df.shape[0]} rows, {df.shape[1]} columns"])
    
    # Number of null values
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        null_cols = null_counts[null_counts > 0]
        summary.append(["Number of null values:", f"{null_counts.sum()} in total across {len(null_cols)} columns"])
    else :
        summary.append(["Number of null values:", "None"])
        
    # Number of duplicates
    num_duplicates = df.duplicated().sum()
    if num_duplicates > 0:
        summary.append(["Number of duplicates:", f"{num_duplicates}"])
    else:
        summary.append(["Number of duplicates:", "None"])
    
    # Columns with many outliers
    outlier_threshold = 3
    outliers_summary = []
    for col in df.select_dtypes(include=np.number).columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        interquartile_range = q3 - q1
        outliers = ((df[col] < q1 - outlier_threshold*interquartile_range) | (df[col] > q3 + outlier_threshold*interquartile_range))
        outlier_percentage = outliers.sum()/df.shape[0]
        if outlier_percentage > 0.05:
            outliers_summary.append([col, f"{outlier_percentage:.2%}"])
    
    if outliers_summary:
        summary.append(["Columns with many outliers:", ""])
        summary.extend(outliers_summary)
        summary.append(["", ""])
    
    # Imbalanced columns
    imbalanced_summary = []
    for col in df.select_dtypes(include=["object", "category"]).columns:
        if df[col].nunique() > 2:
            value_counts = df[col].value_counts(normalize=True)
            if (value_counts < 0.05).any():
                imbalanced_summary.append([col, ""])
    
    if imbalanced_summary:
        summary.append(["Imbalanced categorical columns:", ""])
        summary.extend(imbalanced_summary)
        summary.append(["", ""])
    
    # Categorical columns that need encoding
    cat_cols = df.select_dtypes(include=["object", "category"]).columns
    cat_cols_to_encode = [col for col in cat_cols if len(df[col].value_counts()) > 2]
    if cat_cols_to_encode:
        summary.append(["Categorical columns that need encoding for ML:", ""])
        summary.append([", ".join(cat_cols_to_encode), ""])
        summary.append(["", ""])
    else:
        summary.append(["All categorical columns are encoded for ML.", ""])
        summary.append(["", ""])
    
    # Binary columns that need encoding
    binary_cols = [col for col in df.columns if len(df[col].value_counts()) == 2]
    binary_cols_to_encode = [col for col in binary_cols if df[col].dtype == "object" or df[col].dtype.name == "category"]
    if binary_cols_to_encode:
        summary.append(["Boolean columns that need encoding for ML:", ""])
        summary.append([", ".join(binary_cols_to_encode), ""])
        summary.append(["", ""])
    else:
        summary.append(["All boolean columns are encoded for ML.", ""])
        summary.append(["", ""])
    
    # Columns with only one unique value
    single_value_cols = [col for col in df.columns if len(df[col].value_counts()) == 1]
    if single_value_cols:
        summary.append(["Columns with only one unique value:", ""])
        summary.append([", ".join(single_value_cols), ""])
    
    # Print summary table
    print(tabulate(summary, headers=["Issue", "Details"], tablefmt="pretty"))

dataframe_summary(df)

+--------------------------------------------------------------------+-----------------------+
|                               Issue                                |        Details        |
+--------------------------------------------------------------------+-----------------------+
|                    Number of rows and columns:                     | 1470 rows, 35 columns |
|                       Number of null values:                       |         None          |
|                       Number of duplicates:                        |         None          |
|                    Columns with many outliers:                     |                       |
|                         PerformanceRating                          |        15.37%         |
|                                                                    |                       |
|                  Imbalanced categorical columns:                   |                       |
|                             Department          

In [4]:
# Drop the specified columns
df.drop(columns=["EmployeeCount", "Over18", "StandardHours"], inplace=True)

# Replace "True" and "False" with 1 and 0 in Attrition and OverTime columns
df["Attrition"] = df["Attrition"].replace({"Yes": 1, "No": 0})
df["OverTime"] = df["OverTime"].replace({"Yes": 1, "No": 0})

# Replace "Male" and "Female" with 0 and 1 in Gender column
df["Gender"] = df["Gender"].replace({"Male": 0, "Female": 1})

# Apply one-hot encoding to categorical variables
categorical_columns = ["BusinessTravel", "Department", "EducationField", "JobRole", "MaritalStatus"]
encoder = OneHotEncoder(sparse=False, drop="first")
encoded_columns = encoder.fit_transform(df[categorical_columns])
categories = encoder.categories_
encoded_columns_df = pd.DataFrame(encoded_columns, columns=[f"{col}_{val}" for col, cats in zip(categorical_columns, categories) for val in cats[1:]])
df.drop(columns=categorical_columns, inplace=True)
df = pd.concat([df, encoded_columns_df], axis=1)

# Print the preprocessed dataset
df.head()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,...,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single
0,41,1,1102,1,2,1,2,1,94,3,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,49,0,279,8,1,2,3,0,61,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,37,1,1373,2,2,4,4,0,92,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,33,0,1392,3,4,5,4,1,56,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,27,0,591,2,1,7,1,0,40,3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
df.to_csv("../data/preprocessed_info_satisfaccion_trabajo.csv", index=False)